<div class="usecase-title">Self Designed Walking Tour</div>

<div class="usecase-authors"><b>Authored by: </b> Amy Tran</div>

<div class="usecase-duration"><b>Duration:</b> 60 mins</div>

<div class="usecase-level-skill">
    <div class="usecase-level"><b>Level: </b>Intermediate</div>
    <div class="usecase-skill"><b>Pre-requisite Skills: </b>Python</div>
</div>

<div class="usecase-section-header">Scenario</div>

As international and domestic tourists are returning to Melbourne after Covid, a tool that helps visitors create their own walking experience is beneficial. Based on the visitor’s preference, the tool will suggest appropriate walking route that incorporates landmarks or places of interest. 

<div class="usecase-section-header">What this use case will teach you</div>

At the end of this use case you will:
- Load relevant datasets required for analysis
- Map city landmark

<div class="usecase-section-header">Python Libraries</div>

In [1]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime
import plotly.express as px
import geopy.distance

#pip install plotly==5.8.0
#pip install geopy

<div class="usecase-section-header">Data Extraction</div>

In [3]:
def get_data(base, url, size = 0):
    target_filters = f'records?limit={10}&offset={size}&timezone=UTC'
    target_url = f'{base}{url}/{target_filters}'
    result = session.get(target_url+f'&apikey={API_KEY}')
    status_code = result.status_code
    if status_code == 200:
        result_json = result.json()
        max_results = result_json['total_count']
        links = result_json['links']
        records = result_json['records']
        records_df = pd.json_normalize(records)
    
        #Update column labels
        records_df.drop(columns=['links'],inplace=True)
        column_names = records_df.columns.values.tolist()

        #Replace geolocation.lat & geolocation.lon
        column_names = ['_'.join((a.split(".")[-2:])) if a.split('.')[-2]=='geolocation' else a for a in column_names]
        column_names = [i.split('.')[-1] for i in column_names]
        records_df.columns = column_names
    
        next_url = None
             
        #Obtain next url
        if records_df.shape[0] != max_results:
            for l in links:
                if l['rel'] == 'next':
                    next_url = l['href']
    
        return[records_df, next_url, column_names, max_results, status_code]
    else: return[None, None, None, None, status_code]

In [8]:
#Extract self guided walks data 
session = requests.Session()
base = 'https://data.melbourne.vic.gov.au/api/v2/catalog/datasets/'
url = 'self-guided-walks'

target_url = f'{base}{url}/exports/json'
result = session.get(target_url)
result_json = result.json()
data = pd.json_normalize(result_json)
walks = data.copy()
#Rename Longitude and Latitude columns
walks = walks.rename(columns = {'geo_point_2d.lon' : 'lon', 'geo_point_2d.lat' : 'lat'})
print('Download completed:', walks.shape[0], 'rows,', walks.shape[1], 'columns extracted' )

Download completed: 8 rows, 9 columns extracted


In [9]:
#Extract landmarks and places of interest data 
session = requests.Session()
base = 'https://data.melbourne.vic.gov.au/api/v2/catalog/datasets/'
url = 'landmarks-and-places-of-interest-including-schools-theatres-health-services-spor'

target_url = f'{base}{url}/exports/json'
result = session.get(target_url)
result_json = result.json()
data = pd.json_normalize(result_json)
landmarks = data.copy()
#Rename Longitude and Latitude columns
landmarks = landmarks.rename(columns = {'geo_point_2d.lon' : 'lon', 'geo_point_2d.lat' : 'lat'})
print('Download completed:', landmarks.shape[0], 'rows,', landmarks.shape[1], 'columns extracted' )

Download completed: 242 rows, 5 columns extracted
